# QUANT.PY - 002

* Lives toda terça-feira, 20h.
  
  ### Grupo das lives na descrição:
    * O código/material será enviado lá. Está na descrição do grupo. 
    * Avisos das próximas lives.
    * Cronograma de projetos do mês. Ative o lembrete da nossa próxima live! 
<br>
* Aqui eu vou abordar direto na prática. Se você for muito iniciante e se sentir perdido, te convido a entrar no Finanças com Dados pra aprender toda base do Python.

* Vai rolar um evento chamado Arquivos Confidenciais daqui a 2 semanas onde eu vou mostrar 3 modelos de investimento lucrativos e inéditos. No final do evento vai rolar uma promoção pra entrar no Códigopy, que é minha formação de analise de dados pro mercado financeiro. 2º link da descrição.

----

# Projeto: Qual o melhor ativo para investir com R$ 100 reais?

* Essa análise também serve para valores maiores. 
* Retorno histórico
* Risco histórico
* Retorno ajustado ao Risco (Índice Sharpe)

## Ativos analisados

IBOV, S&P500, QQQ, IFIX, SMALL CAPS BR, IMAB5, CDI, IDIV, SPHD, OURO, BITCOIN.

## Fonte Dados

* Yahoo Finance: BTC, IBOV, SP500, SPHD, GLD, QQQ
* VAROS: IFIX, SMALL CAPS BR, IDIV
* Banco central: CDI e PTAX


### Passo a passo:

   **Passo 1** - Importar bibliotecas
   
   **Passo 2** - Pegar dados do dólar
   
   **Passo 3** - Pegar dados do Yahoo Finance
   
   **Passo 4** - Transformar ativos dolarizados para reais
   
   **Passo 5** - Calcular retornos diários dos ativos do Yahoo Finance
   
   **Passo 6** - Pegar dados de índices fornecidos pela VAROS
   
   **Passo 7** - Pegar o retorno do CDI do banco central
   
   **Passo 8** - Juntar todos os dados de retorno

   **Passo 9** - Calcular volatilidade histórica

   **Passo 10** - Calcular retorno histórico (acumulado e ao ano)

   **Passo 11** - Calcular retorno ajustado ao risco 

   **Passo 12** - Visualizar resultados finais

   ****
   
   
-------------------

In [ ]:
!pip install yfinance

In [ ]:
!pip install python-bcb

# Passo 1 - Importar bibliotecas

In [ ]:
import pandas as pd
import yfinance as yf
from bcb import sgs
from bcb import currency
import datetime
import numpy as np

# Passo 2 - Pegar dados do dólar

In [ ]:
cotacao_dolar = currency.get({"USD": 1}, start= datetime.datetime(2014, 9, 17), end = datetime.datetime.now())

cotacao_dolar

# Passo 3 - Pegar dados do Yahoo Finance

In [ ]:
dados_yahoo_finance = yf.download(["^BVSP", "^GSPC", "SPHD", "GLD", "BTC-USD", "QQQ"], end = "2024-10-01")['Adj Close']
dados_yahoo_finance = dados_yahoo_finance.dropna()
dados_yahoo_finance

# Passo 4 - Transformar ativos dolarizados para reais

In [ ]:
for ativo in dados_yahoo_finance.columns:

    if ativo == "^BVSP":
        pass
    else:
        
        dados_yahoo_finance[ativo] = dados_yahoo_finance[ativo] * cotacao_dolar["USD"]

dados_yahoo_finance.columns = dados_yahoo_finance.columns.str.replace('BTC-USD', 'BTC-BRL')

dados_yahoo_finance

# Passo 5 - Calcular retornos diários dos ativos do Yahoo Finance

In [ ]:
dados_yahoo_finance = dados_yahoo_finance.pct_change().dropna()

dados_yahoo_finance

# Passo 6 - Pegar dados de índices fornecidos pela VAROS

In [ ]:
dados_indices = pd.read_csv("dados_indices_base_varos.csv", index_col = "data")
dados_indices.index = pd.to_datetime(dados_indices.index)

dados_indices

# Passo 7 - Pegar o retorno do CDI do banco central

In [ ]:
rendimento_cdi = sgs.get({"CDI": 12}, start= datetime.datetime(2014, 9, 18))

rendimento_cdi = rendimento_cdi/100

rendimento_cdi

# Passo 8 - Juntar todos os dados de retorno

In [ ]:
dados_totais = dados_yahoo_finance.join(dados_indices)
dados_totais = dados_totais.join(rendimento_cdi)

dados_totais

# Passo 9 - Calcular volatilidade histórica

In [ ]:
vol_historica = dados_totais.std() * np.sqrt(252)

vol_historica.sort_values(ascending = False)

# Passo 10 - Calcular retorno histórico (acumulado e ao ano)

In [ ]:
retorno_acumulado = (1 + dados_totais).cumprod() - 1
retorno_acumulado = retorno_acumulado.iloc[-1, :]

retorno_acumulado

In [ ]:
investiu_100_reais = (1 + dados_totais).cumprod() * 100

investiu_100_reais

In [ ]:
#retorno ao ano

numero_de_dias = len(dados_totais)

retorno_ao_ano = (1 + retorno_acumulado) ** (252/numero_de_dias) - 1

retorno_ao_ano

# Passo 11 - Calcular retorno ajustado ao risco

In [ ]:
sharpe = retorno_ao_ano/vol_historica

sharpe

# Passo 12 - Visualizar resultados finais

In [ ]:
tabela_final = pd.concat([retorno_ao_ano, vol_historica, sharpe], axis = 1)
tabela_final.columns = ["Retorno a.a.", "Volatilidade a.a.", "Índice Sharpe"]
tabela_final.index = ["BITCOIN", "OURO", "QQQ", "DIVIDENDOS US", "IBOV", "S&P500", "IFIX", "SMALL CAPS BR", 
                      "IDIV", "CDI"]
tabela_final = tabela_final.sort_values("Retorno a.a.", ascending = False)

tabela_final

In [ ]:
df_estilizado = tabela_final.style.format({
    'Retorno a.a.': '{:.2%}',
    'Volatilidade a.a.': '{:.2%}'
})

df_estilizado

In [ ]:
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")

In [ ]:
investiu_100_reais.plot()

In [ ]:
investiu_100_reais_ex_btc = investiu_100_reais.drop("BTC-BRL", axis = 1)

In [ ]:
investiu_100_reais_ex_btc.plot()